In [8]:
import pandas as pd
from scraper import Scraper
import os

In [13]:
csv_name = '20_tokopedia_products.csv'

if csv_name in os.listdir():
    data = pd.read_csv(csv_name)
else:
    a = Scraper()
    data = a.get_data()
    a.driver.quit()
    
    data = pd.DataFrame(data)
    data.to_csv(csv_name, index=False)

In [15]:
data

,Prodcut,Price,Description
0,souvenir kado hampers pernikahan wedding gift,Rp49.900,ready stock Kado pernikahan premium \nPENTING=...
1,HAMPERS BAYI | KADO LAHIRAN BAYI / NEWBORN GIF...,Rp83.000,Kado untuk bayi yang praktis dan estetik\nHamp...
2,FLUX SET - Wedding Gift | Birthday Hampers | K...,Rp159.000,FLUX SET :\n- RTR : Classic Mini Air Diffuser ...
3,hampers mug couple wedding gift,Rp85.000,"jika ingin d berikan kartu ucapan, mohon tuli..."
4,Gift Box Ulang Tahun / Kado Cewek Ultah Murah ...,Rp50.000,untuk order jangan lupa kirim format order (un...
5,Hampers Bayi Baby Girl Boy Kado Lahiran Peremp...,Rp110.000,Kado Lahiran Bayi \nDijamin nyaman dan hangat ...
6,Hampers Baby Boy Gift Set Parcel Kado Bayi Lak...,Rp88.000,"Ready Stok, Siap Kirim. \nMohon Baca Deskripsi..."
7,Newborn Hampers Gift Set | Baby Hampers | Hamp...,Rp365.000,NEW PREMIUM BABY GIFT SET\n• Premium Bunny Dol...
8,Souvenir Mangkok Set Keramik Handpainted Japan...,Rp29.999,Mangkuk Jepang Set dengan sumpit bambu / Japan...
9,Souvenir Mangkok Set Mangkok Keramik Handpaint...,Rp27.999,Mangkuk Jepang Set dengan sumpit bambu / Japan...
